In [1]:
import keras

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

def encode(series):
    return pd.get_dummies(series.astype(str))

Using TensorFlow backend.


In [2]:
cnnr_model = keras.models.load_model('cnnr_iter1.h5')
cnnr_model.summary()
cnnc_model = keras.models.load_model('cnnc_iter3.h5')
cnnc_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 2)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 12)        228       
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 12)        48        
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 108)         32508     
_______________________________________

In [3]:
from keras.models import Model
dense_modelc = Model(inputs=cnnc_model.input,outputs=cnnc_model.layers[11].output)
dense_modelr = Model(inputs=cnnr_model.input,outputs=cnnr_model.layers[11].output)

In [22]:
#Import labeled data

temp_labels = np.loadtxt('vec_mat_clabels_icex_src_0.01train.csv',delimiter=",")
labels_t = []

for l in range(len(temp_labels)):
    labels_t.append(str(temp_labels[l]))

labels_t = np.array(labels_t)
labels_t = labels_t.ravel()

def encode(series):
    return pd.get_dummies(series.astype(str))

y_train = encode(labels_t)
labels = list(y_train.columns.values)
float_labels = [float(i) for i in labels_t]

#Import test data
features_test = np.loadtxt('vec_mat_features_icex_src_0.01train_norm.csv',delimiter=",")
temp_ytest = np.loadtxt('vec_mat_rlabels_icex_src_0.01train.csv',delimiter=",")
y_test= []

real_test = features_test[:,0::2]
imag_test = features_test[:,1::2]
X_test = np.zeros((features_test.shape[0],32,32,2))

for k in range(features_test.shape[0]):
    count = 0
    for i in range(32):
        for j in range(i,32):
            X_test[k,i,j,0] = real_test[k,count]
            X_test[k,i,j,1] = imag_test[k,count]
            
            if i!=j:
                X_test[k,j,i,0] = X_test[k,i,j,0]
                X_test[k,j,i,1] = -X_test[k,i,j,1]
            
            count = count + 1



temp_ytest = temp_ytest.ravel()
y_test = temp_ytest

ind = np.where(y_test==33.0)

In [23]:
x = X_test[ind[0][0],:,:,:]
x = x[None,:,:,:]
layerd_outputc_train = dense_modelc.predict(X_test)
layerd_outputr_train = dense_modelr.predict(X_test)

print(np.shape(layerd_outputc_train))
print(np.shape(layerd_outputr_train))


(4701, 256)
(4701, 256)


In [24]:
np.savetxt('layerd_outputc_train.tsv',layerd_outputc_train,delimiter='\t')
np.savetxt('layerd_outputr_train.tsv',layerd_outputr_train,delimiter='\t')